In [1]:
using PLite

┌ Info: Precompiling PLite [7876af07-990d-54b4-8888-23690620f79a]
└ @ Base loading.jl:1317
┌ Warning: Package PLite does not have SharedArrays in its dependencies:
│ - If you have PLite checked out for development and have
│   added SharedArrays as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with PLite
└ Loading SharedArrays into PLite from project dependency, future warnings for PLite are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [2]:
# constants
const MinX = 0
const MaxX = 100
const StepX = 20

# mdp definition
mdp = MDP()

MDP(Dict{AbstractString, PLite.LazyVar}(), Dict{AbstractString, PLite.LazyVar}(), PLite.LazyFunc(true, String[], PLite.var"#emptyfunc#1"()), PLite.LazyFunc(true, String[], PLite.var"#emptyfunc#1"()))

In [3]:
# state space
statevariable!(mdp, "x", MinX, MaxX)  # continuous
statevariable!(mdp, "goal", ["no", "yes"])  # discrete

# action space
actionvariable!(mdp, "move", ["W", "E", "stop"])  # discrete

PLite.ValuesVar("move", ["W", "E", "stop"])

In [4]:
function isgoal(x::Float64)
  if abs(x - MaxX / 2) < StepX
    return "yes"
  else
    return "no"
  end
end

function mytransition(x::Float64, goal::AbstractString, move::AbstractString)
  if isgoal(x) == "yes" && goal == "yes"
    return [([x, isgoal(x)], 1.0)]
  end

  if move == "E"
    if x >= MaxX
      return [
        ([x, isgoal(x)], 0.9),
        ([x - StepX, isgoal(x - StepX)], 0.1)]
    elseif x <= MinX
      return [
        ([x, isgoal(x)], 0.2),
        ([x + StepX, isgoal(x + StepX)], 0.8)]
    else
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.1),
        ([x + StepX, isgoal(x + StepX)], 0.8)]
    end
  elseif move == "W"
    if x >= MaxX
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.9)]
    elseif x <= MinX
      return [
        ([x, isgoal(x)], 0.9),
        ([x + StepX, isgoal(x + StepX)], 0.1)]
    else
      return [
        ([x, isgoal(x)], 0.1),
        ([x - StepX, isgoal(x - StepX)], 0.8),
        ([x + StepX, isgoal(x + StepX)], 0.1)]
    end
  elseif move == "stop"
    return [([x, isgoal(x)], 1.0)]
  end
end

mytransition (generic function with 1 method)

In [5]:
transition!(mdp, ["x", "goal", "move"], mytransition)

PLite.LazyFunc(false, ["x", "goal", "move"], mytransition)

In [6]:
function myreward(x::Float64, goal::AbstractString, move::AbstractString)
  if goal == "yes" && move == "stop"
    return 1
  else
    return 0
  end
end

reward!(mdp, ["x", "goal", "move"], myreward)

PLite.LazyFunc(false, ["x", "goal", "move"], myreward)

In [7]:
solver = SerialValueIteration()

SerialValueIteration(true, 1000, 0.0001, 0.99, Dict{AbstractString, PLite.LazyDiscrete}(), Dict{AbstractString, PLite.LazyDiscrete}(), GridInterpolations.RectangleGrid{0}(), GridInterpolations.RectangleGrid{0}())

In [8]:
const StepX = 20
discretize_statevariable!(solver, "x", StepX)

PLite.LazyDiscrete("x", 20.0)

In [9]:
solver = SerialValueIteration(maxiter=40, discount=1, verbose=true)
discretize_statevariable!(solver, "x", StepX)

PLite.LazyDiscrete("x", 20.0)

In [11]:
# solver options
solver = SerialMCTS()
discretize_statevariable!(solver, "x", StepX)

# generate results
solution = solve(mdp, solver)

┌ Info: mdp and monte-carlo tree search solver passed basic checks
└ @ PLite /Users/jeff/.julia/packages/PLite/ud1HP/src/mcts_checks.jl:5


SerialMCTS(50, 20, 3.0, 0.99, Dict{Vector{T} where T, PLite.StateNode}(), Dict{AbstractString, PLite.LazyDiscrete}("x" => PLite.LazyDiscrete("x", 20.0)), Dict{AbstractString, PLite.LazyDiscrete}(), GridInterpolations.RectangleGrid{2}([0.0, 20.0, 40.0, 60.0, 80.0, 100.0],[1.0, 2.0],), GridInterpolations.RectangleGrid{1}([1.0, 2.0, 3.0],))

In [12]:
policy = getpolicy(mdp, solution)

(::PLite.var"#policy#23"{MDP, SerialMCTS}) (generic function with 1 method)

In [13]:
stateq = (20.0, "no")
actionq = policy(stateq...)

LoadError: MethodError: no method matching -(::Vector{Float64}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar
[0mClosest candidates are:
[0m  -(::Array, [91m::SparseArrays.AbstractSparseMatrixCSC[39m) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/SparseArrays/src/sparsematrix.jl:1747
[0m  -([91m::Base.TwicePrecision[39m, ::Number) at twiceprecision.jl:281
[0m  -([91m::LinearAlgebra.UniformScaling[39m, ::Number) at /Users/julia/buildbot/worker/package_macos64/build/usr/share/julia/stdlib/v1.6/LinearAlgebra/src/uniformscaling.jl:147
[0m  ...